In [6]:
import pandas
import pickle
import itertools
import re
import numpy as np
from pg import DB
db = DB()

In [7]:
protein_to_sequence = pickle.load( open( "files/intermediates/proteins_to_sequences.pkl", "rb" ) )
distanceMatrix_df = pandas.read_pickle(open( "files/intermediates/distanceMatrix.pkl", "rb" ))

In [8]:
proteins = list(protein_to_sequence.keys())
probability_frequency = [len(sequence) for sequence in protein_to_sequence.values()]
probability_distribution = [freq/sum(probability_frequency) for freq in probability_frequency]
sum(probability_distribution)

1.000000000000001

Only get samples for the sizes we need. In this case instead of doing samples for 1381 different sizes we end up only needed smaples for 730 different sizes

In [9]:
sizes_of_protein_sets_in_data_set = db.query("SELECT proteins_length FROM motif").getresult()
sizes_of_protein_sets_in_shuffled_data_set = db.query("SELECT shuffled_proteins_length FROM motif").getresult()
sizes_of_protein_sets_unique = set(sizes_of_protein_sets_in_data_set+sizes_of_protein_sets_in_shuffled_data_set)
df = pandas.DataFrame(list(sizes_of_protein_sets_unique))
df.describe()#metadata about sizes

,0
count,730.000000
mean,397.178082
std,266.739792
min,2.000000
25%,184.250000
50%,366.500000
75%,552.500000
max,1381.000000


In [10]:
for size in sorted(sizes_of_protein_sets_unique):
    print(size)

(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)
(11,)
(12,)
(13,)
(14,)
(15,)
(16,)
(17,)
(18,)
(19,)
(20,)
(21,)
(22,)
(23,)
(24,)
(25,)
(26,)
(27,)
(28,)
(29,)
(30,)
(31,)
(32,)
(33,)
(34,)
(35,)
(36,)
(37,)
(38,)
(39,)
(40,)
(41,)
(42,)
(43,)
(44,)
(45,)
(46,)
(47,)
(48,)
(49,)
(50,)
(51,)
(52,)
(53,)
(54,)
(55,)
(56,)
(57,)
(58,)
(59,)
(60,)
(61,)
(62,)
(63,)
(64,)
(65,)
(66,)
(67,)
(68,)
(69,)
(70,)
(71,)
(72,)
(73,)
(74,)
(75,)
(76,)
(77,)
(78,)
(79,)
(80,)
(81,)
(82,)
(83,)
(84,)
(85,)
(86,)
(87,)
(88,)
(89,)
(90,)
(91,)
(92,)
(93,)
(94,)
(95,)
(96,)
(97,)
(98,)
(99,)
(100,)
(101,)
(102,)
(103,)
(104,)
(105,)
(106,)
(107,)
(108,)
(109,)
(110,)
(111,)
(112,)
(113,)
(114,)
(115,)
(116,)
(117,)
(118,)
(119,)
(120,)
(121,)
(122,)
(123,)
(124,)
(125,)
(126,)
(127,)
(128,)
(129,)
(130,)
(131,)
(132,)
(133,)
(134,)
(135,)
(136,)
(137,)
(138,)
(139,)
(140,)
(141,)
(142,)
(143,)
(144,)
(145,)
(146,)
(147,)
(148,)
(149,)
(150,)
(151,)
(152,)
(153,)
(154,)
(155,)
(156,)
(157,)
(158,)
(159,)


In [ ]:
30 sec for size 10 with 10k samples
20 min for size 100 with 10k samples
80 min for size 200 with 10k samples
250**2/200**2 = 1.56
80*1.56*1.1 = 137
300**2/200**2 = 2.25
80*2.25*1.1 = 198
350**2/200**2 = 3.06
80*3.06*1.1 = 270
400**2/200**2 = 4
80*4*1.1 = 352
prob 200min-2000min for size 1k with 10k samples

In [ ]:
10 min for 0-50
30 min for 50-100
90 for 100-150
120 for 150-200
140 for 200-250
200 for 250-300
270 for 300-350
352 for 350-400
should be done by 5pm

The below will run on the cluster

In [45]:
%%time
import pandas
import pickle
import numpy as np
from pg import DB

protein_to_sequence = pickle.load( open( "files/intermediates/proteins_to_sequences.pkl", "rb" ) )
distanceMatrix_df = pandas.read_pickle(open( "files/intermediates/distanceMatrix.pkl", "rb" ))

# print(sys.argv[1])
size = 50

number_of_samples_needed = 10000
samples_scores = {"average_scores" : [], "tt_average_scores" :[]}

for i in range(0, number_of_samples_needed):
    sample = np.random.choice(proteins, size, p = probability_distribution, replace = False)
    sample_subgraph = []
    for protein_i in sample:#original subgraph
        row = []
        for protein_j in sample:
            if(protein_i != protein_j):
                row.append(distanceMatrix_df[protein_i][protein_j])
        sample_subgraph.append(row)
    avg_score = pandas.DataFrame(sample_subgraph).mean().mean()
    ninety_p_shuffled = np.percentile(sample_subgraph, 90)
    for i, row in enumerate(sample_subgraph):
        for j, elem in enumerate(row):
            if(elem < ninety_p_shuffled):
                sample_subgraph[i][j] = None
                
    tt_avg_score = pandas.DataFrame(sample_subgraph).mean().mean()
#     print(avg_score, tt_avg_score)
    samples_scores["average_scores"].append(avg_score)
    samples_scores["tt_average_scores"].append(tt_avg_score)
pickle.dump( samples_scores,open( "files/intermediates/samples/sample_" + str(size) + ".pkl", "wb" ) )

CPU times: user 5min 12s, sys: 464 ms, total: 5min 13s
Wall time: 5min 13s


In [42]:
pandas.DataFrame(samples_scores)

sample = pandas.DataFrame(pandas.read_pickle(open( "files/intermediates/samples/sample_" + str(size) + ".pkl", "rb" )))
sample.describe()

,average_scores,tt_average_scores
count,10000.000000,10000.000000
mean,0.026660,0.428946
std,0.005402,0.030440
min,0.010175,0.327627
25%,0.022832,0.407653
50%,0.026224,0.428342
75%,0.029977,0.448694
max,0.054830,0.549130


In [ ]:
%%time
number_of_samples_needed = 10000
for size_t in sorted(sizes_of_protein_sets_unique):
    size = size_t[0]
    number_of_samples_taken_for_size = db.query("Select count(*) from random_samples_average_score where size = '" + str(size) +"'").getresult()[0][0]
    while(number_of_samples_taken_for_size < number_of_samples_needed):
        sample = np.random.choice(proteins, size, p = probability_distribution, replace = False)
        sample_subgraph = []
        for protein_i in sample:#original subgraph
            row = []
            for protein_j in sample:
                if(protein_i != protein_j):
                    row.append(distanceMatrix_df[protein_i][protein_j])
            sample_subgraph.append(row)
        avg_score = pandas.DataFrame(sample_subgraph).mean().mean()
        ninety_p_shuffled = np.percentile(sample_subgraph, 90)
        for i, row in enumerate(sample_subgraph):
            for j, elem in enumerate(row):
                if(elem < ninety_p_shuffled):
                    sample_subgraph[i][j] = None
        tt_avg_score = pandas.DataFrame(sample_subgraph).mean().mean()
        
        do_not_print = db.insert('random_samples_average_score',
                         size = size,
                         average_score = avg_score,
                         tt_average_score = tt_avg_score

                        )
        number_of_samples_taken_for_size = db.query("Select count(*) from random_samples_average_score where size = '" + str(size) +"'").getresult()[0][0]



ABCB7 198
DHX30 76
SLC30A9 1408
KIAA0391 512
AFG3L2 621
GLUD1 1549
NDUFAF4 1793
ATPAF1 737
ATP5B 148
ETFA 271
IARS2 406
MDH2 202
OAT 639
ERAL1 501
PPIF 1491
MTHFD1L 393
NDUFV3 642
LETM1 2945
NDUFAF2 82
ACOT1 139
LRPPRC 2376
MRPS31 76
SHMT2 574
EARS2 2358
TACO1 363
MTHFD2 1055
MRPL48 241
QRSL1 2415
GTPBP10 6301
GRSF1 5105
MRPS14 1711
MRPL45 623
MRPS23 857
RBFA 4715
NGRN 423
MRPL10 933
MRPL42 2551
OXSM 40
MTRF1L 2630
HSPE1 204
BCS1L 57
C8orf82 1123
MRPS17 162
NDUFA6 657
CDK5RAP1 204
TRMT61B 377
HSDL2 1861
MRPS36 933
FECH 5871
PDE12 2037
SUCLA2 715
NME4 412
MRPL46 135
C17orf80 1615
MRPS24 141
TIMM44 469
GLS 2558
SDHA 279
METTL15 2666
MRPS28 120
NFS1 934
POLG 446
MMAB 3312
TEFM 186
SUPV3L1 58
NIPSNAP1 1126
THNSL1 1209
ABHD10 1647
MRPL24 96
ATPIF1 1649
NDUFS6 137
FLAD1 98
PREPL 3747
DHTKD1 2337
PNPLA8 1979
PDK1 3165
CARS2 127
RTN4IP1 968
PNPT1 2174
ALAS1 147
PPA2 648
GRPEL1 809
NDUFA12 112
PTCD3 4598
PYCR1 821
MRPS26 381
FASTKD5 257
ACOT9 234
ACADSB 4525
MRPL44 680
ATP5J2-PTCD1 3228
FASTKD2

XXYLT1 1431
PCYOX1L 1004
OS9 648
NCSTN 681
LMAN2 524
SEP15 1102
SDF2L1 124
TTC17 1029
B4GALT3 506
SDF2 633
LGALS3BP 323
COL6A1 1036
FBN2 1546
VLDLR 617
GUSB 211
NID1 2035
CTAGE5 929
SEC11A 482
SPCS3 3915
PIGK 3392
ERLIN1 2041
ERLIN2 3686
PIGT 419
SLC35A3 4502
TMED4 316
GLG1 5739
GOLIM4 403
PDIA4 732
QSOX2 2392
B3GALT6 1772
PRDX4 62
CANX 2994
RPN2 307
SERPINH1 721
NCLN 1887
IKBIP 903
ASPH 2563
TMED2 1412
UGGT1 5933
P4HB 854
PRKCSH 338
NUCB2 104
SDF4 762
ERAP2 2097
NENF 373
HSPA5 1744
HSP90B1 268
HYOU1 1409
DNAJC10 2891
MLEC 5319
TOR1A 1030
DDOST 612
NUP210 1444
GXYLT1 5940
GANAB 989
DNAJC3 3959
DNAJB11 392
ERP29 1053
RCN2 1038
LAMB1 150
KDELC2 2721
RCN1 1254
CNPY2 326
MESDC2 3465
CALU 4256
LRPAP1 9377
TMTC3 4238
CALR 577
PPIB 208
PDIA3 1388
PDIA6 867
TXNDC5 1894
TRABD 1079
USP30 2095
OCIAD1 1121
MAVS 9976
ATAD1 1569
DNM1L 2240
ABCD3 1534
MARC2 367
CISD1 1609
FKBP8 411
MLX 1600
FAM213A 1634
FUNDC2 2244
ARMC10 1210
HK1 746
TOMM20 2721
CDC23 1330
FBXL4 553
RHOT2 568
EXD2 2920
TDRKH 461
BNI

COPA 1570
ARCN1 2318
COG3 1967
TMF1 3350
COG4 441
BSDC1 1518
SCOC 1452
KIAA1033 2197
EPHB3 786
USP8 1992
VPS54 897
GORAB 1371
NEO1 2510
SNX17 814
RAB5C 807
IL6ST 5985
AP1AR 1646
PSTPIP2 1923
PPP1R21 648
TECPR2 367
FAM160A2 172
FAM160A1 878
APPL1 3783
VPS11 355
APPL2 1017
GDI2 783
SZT2 3767
SLCO4C1 2870
ARHGEF10 1378
PIK3C2B 2222
RRAGA 422
PRKAA2 7622
PRKAA1 3399
PRKAB1 1299
TECPR1 1947
PLEKHA8 568
DEPDC5 537
RRAGB 431
LAMTOR2 139
PRKAG1 627
CKB 206
SH2B2 81
RNFT1 615
CCDC116 257
EDNRA 2336
AP1M1 910
ITGA1 1204
ATAD3C 1628
MSH5 147
VPS52 569
STX16 3235
CHM 2473
GPR108 398
TBC1D25 1634
SUSD5 2697
NR3C1 1574
PCLO 1998
MYO1C 1540
TUBB2A 195
EPPK1 247
STK39 1476
IPO11 1271
GSG2 366
URB2 902
TRMT1L 1933
TAF2 1150
CHD2 153
SATB2 2711
ZFHX3 4277
DOT1L 2786
CIC 606
HIVEP1 538
BCOR 863
GLTSCR1 862
BCL11A 3210
SALL1 1137
MLLT6 4205
TFAP2A 1734
ANKRD11 857
ZBTB10 2847
TCF20 1328
ZNF536 894
CASZ1 591
ERCC6 2370
KAT6A 2726
CHD6 2063
HAT1 334
RBBP7 383
GTF3C5 523
GTF3C3 1531
GTF3C1 737
PSME3 2182
WBP

In [14]:

db.query("""create table random_samples_average_score(
         size integer,
         average_score float,
         tt_average_score float
        )""")

In [13]:
# db.query("DROP TABLE random_samples_average_score")

In [117]:
size = 4
db.query("Select count(size) from random_samples where size = '" + str(size) +"'").getresult()[0][0]

1

In [116]:
db.query("Select count(size) from random_samples").getresult()

[(451,)]

In [103]:
size = 2
number_of_samples_taken_for_size = db.query("Select count(*) from random_samples where size = '" + str(size) +"'").getresult()[0][0]
number_of_samples_needed = 1
number_of_samples_taken_for_size < number_of_samples_needed


False